In [ ]:
scaling 해보기
adb + voting + stk

In [1]:
import numpy as np
import pandas as pd

from scipy.stats import norm, skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

from sklearn import feature_selection
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.svm import SVR


import tensorflow as tf
from tensorflow import keras
from keras import layers, models, optimizers
from keras.regularizers import L1L2

import os
import time
import gc


Using TensorFlow backend.


In [2]:
training = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

sub = pd.read_csv('./data/sample_submission.csv')
# making copies of original datasets for rest of this kernel
df_train = training.copy()
df_test = test.copy()
print(df_train.shape, df_test.shape)

(1460, 81) (1459, 80)


In [3]:
target = df_train['SalePrice']  #target variable
df_train = df_train.drop('SalePrice', axis=1) 

print("Training: {}, Target: {}, Test: {}".format(df_train.shape, target.shape, df_test.shape))

Training: (1460, 80), Target: (1460,), Test: (1459, 80)


In [4]:
target_tr = np.log1p(target)

In [5]:
def house_pipeline_v1(dataframe,
                      impute_method = "median",
                      feature_transform = "yes",
                      feature_scaling = "RobustScaler", 
                      feature_selection = "yes"):
    # 0. initialising dataframe
    df_pipe = dataframe.copy()
    print("Dataframe loaded.")
    
    # Drop redundant columns
    df_pipe.drop(['Id'], axis=1, inplace=True) # drop Id column
    print("Dropped redundant column 'Id'.")

    # column types variables
    numeric_features = list(df_pipe.select_dtypes(
        include=[np.number]).columns.values)
    categ_features = list(df_pipe.select_dtypes(
        include=['object']).columns.values)
    for col in numeric_features:
        df_pipe[col] = df_pipe[col].astype(float)

    # 1. Handling missing values
    # replacing NaNs in categorical features with "None"
    df_pipe[categ_features] = df_pipe[categ_features].apply(
        lambda x: x.fillna("None"), axis=0)

    # imputing numerical features
    for col in ("LotFrontage", 'GarageYrBlt', 'GarageArea', 'GarageCars'):
        df_pipe[col].fillna(0.0, inplace=True)
        
    if impute_method == "median": # replacing NaNs in numerical features with the median
        df_pipe[numeric_features] = df_pipe[numeric_features].apply(
            lambda x: x.fillna(x.median()), axis=0)
        print("Missing values imputed with median.")
    
    elif impute_method == "mean": # replacing NaNs in numerical features with the mean
        df_pipe[numeric_features] = df_pipe[numeric_features].apply(
            lambda x: x.fillna(x.mean()), axis=0)
        print("Missing values imputed with mean.")

    # 2. Feature Engineering
    # Examples: Discretize Continous Feature;
    #           Decompose Features;
    #           Add Combination of Feature
    df_pipe['YrBltAndRemod']=df_pipe['YearBuilt']+df_pipe['YearRemodAdd']
    df_pipe['TotalSF']=df_pipe['TotalBsmtSF'] + df_pipe['1stFlrSF'] + df_pipe['2ndFlrSF']

    df_pipe['Total_sqr_footage'] = (df_pipe['BsmtFinSF1'] + df_pipe['BsmtFinSF2'] +
                                     df_pipe['1stFlrSF'] + df_pipe['2ndFlrSF'])

    df_pipe['Total_Bathrooms'] = (df_pipe['FullBath'] + (0.5 * df_pipe['HalfBath']) +
                                   df_pipe['BsmtFullBath'] + (0.5 * df_pipe['BsmtHalfBath']))

    df_pipe['Total_porch_sf'] = (df_pipe['OpenPorchSF'] + df_pipe['3SsnPorch'] +
                                  df_pipe['EnclosedPorch'] + df_pipe['ScreenPorch'] + 
                                 df_pipe['WoodDeckSF'])
    print("Feature enginering: added combination of features.")
    
    df_pipe['haspool'] = df_pipe['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
    df_pipe['has2ndfloor'] = df_pipe['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
    df_pipe['hasgarage'] = df_pipe['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
    df_pipe['hasbsmt'] = df_pipe['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
    df_pipe['hasfireplace'] = df_pipe['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)
    print("Feature enginering: added boolean features.")
    
    # 3. Feature Transformations (log(x), sqrt(x), x^2, etc.)
    # Transform numerical features that should be considered as strings 
    df_pipe['MSSubClass'] = df_pipe['MSSubClass'].apply(str)
    df_pipe['YrSold'] = df_pipe['YrSold'].astype(str)
    df_pipe['MoSold'] = df_pipe['MoSold'].astype(str)
    df_pipe['YrBltAndRemod'] = df_pipe['YrBltAndRemod'].astype(str)
    print("Transformed numerical features that should be considered as strings.")
    
    numeric_features = list(df_pipe.select_dtypes(
        include=[np.number]).columns.values)
    categ_features = list(df_pipe.select_dtypes(
        include=['object']).columns.values)
    
    if feature_transform == "yes":
        # Transform all numerical columns with skewness factor > 0.5
        skew_features = df_pipe[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)
        high_skew = skew_features[skew_features > 0.5]
        skew_index = high_skew.index
        for i in skew_index:
            df_pipe[i] = boxcox1p(df_pipe[i], boxcox_normmax(df_pipe[i]+1))
        print("Transformed numerical columns with high skewness factor.")
    elif feature_transform == "no":
        pass

    # 4. Label Encoding
    df_pipe = pd.get_dummies(df_pipe)
    print("Label Encoding: from {} cols to {} cols.".format(
        dataframe.shape[1], df_pipe.shape[1]))

    # 5. Feature Scaling
    #cols = df_pipe.select_dtypes([np.number]).columns
    if feature_scaling == 'MinMaxScaler':
        scaler = MinMaxScaler(feature_range=(0, 1))
        for col in numeric_features:
            df_pipe[[col]] = scaler.fit_transform(df_pipe[[col]])
        print("Performed feature Scaling with MinMaxScaler.")

    elif feature_scaling == 'StandardScaler':
        scaler = StandardScaler()
        for col in numeric_features:
            df_pipe[[col]] = scaler.fit_transform(df_pipe[[col]])
        print("Performed feature Scaling with StandardScaler.")

    elif feature_scaling == "RobustScaler":
        scaler = RobustScaler()
        for col in numeric_features:
            df_pipe[[col]] = scaler.fit_transform(df_pipe[[col]])
        print("Performed feature Scaling with RobustScaler.")
    
    # 6. Feature Selection
    ## let's remove columns with little variance (to reduce overfitting)
    overfit = []
    for i in df_pipe.columns:
        counts = df_pipe[i].value_counts()
        zeros = counts.iloc[0]
        if zeros / len(df_pipe) * 100 > 99.9: # the threshold is set at 99.9%
            overfit.append(i)
    overfit = list(overfit)
    # let's make sure to keep data processing columns needed later on
    try:
        overfit.remove('Dataset_Train')
        overfit.remove('Dataset_Test')
    except:
        pass
    df_pipe.drop(overfit, axis=1, inplace=True)
    print("To prevent overfitting, {} columns were removed.".format(len(overfit)))
    
    ## Summary
    print("Shape of transformed dataset: {} (original: {})".format(df_pipe.shape, dataframe.shape))
    return df_pipe

In [6]:
def target_transf(target, 
                  transform="log"):
    
    if transform == "log":
        target_tranf = np.log1p(target)
        print("Target feature transformed with natural logarithm.")
    
    elif transform == "sqrt":
        target_tranf = np.sqrt(target)
        print("Target feature transformed with sqrt.")
    
    elif transform == "square":
        target_tranf = np.square(target)
        print("Target feature transformed with square.")
    
    print("Shape of transformed target: {}".format(target_tr.shape))
    return target_tranf

In [7]:
# Test pipeline
df_train_test = house_pipeline_v1(df_train)
print("\n")
target_tr = target_transf(target)

Dataframe loaded.
Dropped redundant column 'Id'.
Missing values imputed with median.
Feature enginering: added combination of features.
Feature enginering: added boolean features.
Transformed numerical features that should be considered as strings.


c:\users\young\anaconda3\envs\study\lib\site-packages\scipy\stats\stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\users\young\anaconda3\envs\study\lib\site-packages\scipy\stats\stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


Transformed numerical columns with high skewness factor.
Label Encoding: from 80 cols to 505 cols.
Performed feature Scaling with RobustScaler.
To prevent overfitting, 35 columns were removed.
Shape of transformed dataset: (1460, 470) (original: (1460, 80))


Target feature transformed with natural logarithm.
Shape of transformed target: (1460,)


In [8]:
# let's check that we no longer have any missing values
perc_na = (df_train_test.isnull().sum()/len(df_train_test))*100
ratio_na = perc_na.sort_values(ascending=False)
missing_data = pd.DataFrame({'missing_ratio' :ratio_na})
missing_data = missing_data.drop(missing_data[missing_data.missing_ratio == 0].index)
missing_data.head(5)

,missing_ratio


In [9]:
df_train_pipeline = df_train.copy()
df_test_pipeline = df_test.copy()
# Concat dataframes
df_train_pipeline["Dataset"] = "Train"
df_test_pipeline["Dataset"] = "Test"
# Concat dataframes
df_joined = pd.concat([df_train_pipeline, df_test_pipeline], 
                      sort=False)
df_joined = df_joined.reset_index(drop=True) # reset index
print("Joined Dataframe shape: {}".format(df_joined.shape))

Joined Dataframe shape: (2919, 81)


In [10]:
df_joined_ml = house_pipeline_v1(df_joined,
                                 impute_method = "median",
                                 feature_transform = "yes",
                                 feature_scaling = "RobustScaler", 
                                 feature_selection = "yes")
print("----\n")
target_ml = target_transf(target)
print("----\n")
print("Transformed Joined Dataframe shape: {}, and target shape: {}".format(
    df_joined_ml.shape, target_ml.shape))

Dataframe loaded.
Dropped redundant column 'Id'.
Missing values imputed with median.
Feature enginering: added combination of features.
Feature enginering: added boolean features.
Transformed numerical features that should be considered as strings.


c:\users\young\anaconda3\envs\study\lib\site-packages\scipy\stats\stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\users\young\anaconda3\envs\study\lib\site-packages\scipy\stats\stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


Transformed numerical columns with high skewness factor.
Label Encoding: from 81 cols to 521 cols.
Performed feature Scaling with RobustScaler.
To prevent overfitting, 44 columns were removed.
Shape of transformed dataset: (2919, 477) (original: (2919, 81))
----

Target feature transformed with natural logarithm.
Shape of transformed target: (1460,)
----

Transformed Joined Dataframe shape: (2919, 477), and target shape: (1460,)


In [11]:
df_train_ml = df_joined_ml[df_joined_ml['Dataset_Train']==1].copy()
# Remove redundant features
df_train_ml.drop(['Dataset_Train'], axis=1, inplace=True)
df_train_ml.drop(['Dataset_Test'], axis=1, inplace=True)
# Reset index
df_train_ml = df_train_ml.reset_index(drop=True) 
print(df_train_ml.shape)

(1460, 475)


In [12]:
# Extract Testing data from joined transformed dataset
df_test_ml = df_joined_ml[df_joined_ml['Dataset_Test']==1].copy()
# Remove redundant features
df_test_ml.drop(['Dataset_Train'], axis=1, inplace=True)
df_test_ml.drop(['Dataset_Test'], axis=1, inplace=True)
# Reset index
df_test_ml = df_test_ml.reset_index(drop=True)
print(df_test_ml.shape)

(1459, 475)


In [13]:
train_X = df_train_ml.copy()
target = target_ml.copy()
test_X = df_test_ml.copy()

n, s = np.mean(target), np.std(target)
t = (target - n)/s

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train_ml,
                                                    target_ml,
                                                    test_size=0.2,
                                                    stratify=df_train_ml['OverallQual'],
                                                    random_state=42)

In [ ]:
n_layers = 256
n_node = 3

inputs = layers.Input(shape =(train_X.shape[1], ))

x = layers.Dense(n_layers)(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

for i in range(n_node):
    x = layers.Dense(n_layers//(2**(i+1)), kernel_regularizer=L1L2(l2=0.001), kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.1)(x)

outputs = layers.Dense(1, activation='linear')(x)

m = models.Model(inputs, outputs)

m.compile(optimizer = 'adam',
            loss = 'mse')

In [ ]:
es = keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

In [ ]:
m.fit(train_X, t, 
      epochs = 1000,
     validation_split=0.2,
     callbacks = [es])

In [ ]:
k = m.predict(test_X)

In [ ]:
ans = np.exp(k*s+n)

In [ ]:
# n, s = np.mean(target), np.std(target)
# t = (target - n)/s

In [ ]:
ans

In [ ]:
sub['SalePrice'] = ans

In [ ]:
sub.to_csv('./sub/sub19.csv', index=False)

In [48]:
rf = RandomForestRegressor(n_estimators=1200,
                               max_depth=15,
                               min_samples_split=5,
                               min_samples_leaf=5,
                               max_features=None,
                               oob_score=True,
                               random_state=42)
gbr = GradientBoostingRegressor(n_estimators=6000,
                                learning_rate=0.01,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=15,
                                min_samples_split=10,
                                loss='huber',
                                random_state=42)
svr = SVR()

adb = AdaBoostRegressor(n_estimators=200,
                       base_estimator=gbr,
                       learning_rate=0.01,
                       random_state=42
                       )

elastic = ElasticNet()

vreg = VotingRegressor(estimators=[
    ('rf', rf),
    ('gbr', gbr),
    ('svr', svr),
    ('adb', adb)
],  n_jobs=-1
)

stk_reg = StackingRegressor(estimators=[
    ('rf', rf),
    ('gbr', gbr),
    ('svr', svr),
    ('adb', adb)
],  n_jobs=-1,
#     final_estimator=rf
)



models = [rf, gbr, svr, adb, elastic]

In [ ]:
model = adb
model.fit(train_X, t)
k = model.predict(test_X)

In [ ]:
ans = np.exp(k*s+n)
# ans = np.exp(k)

In [ ]:
ans

In [ ]:
sub['SalePrice'] = ans
sub.to_csv('./sub/sub30.csv', index=False)